In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "password")

with GraphDatabase.driver(URI, auth=AUTH) as driver: 
    driver.verify_connectivity() 

In [4]:
def create_node(driver, name, id, paper_count):
    def create_person(tx, name, id, paper_count):
        query = (
            "CREATE (p:Concept {name: $name, id: $id, paper_count: $paper_count})"
        )
        tx.run(query, name=name, id=id, paper_count=paper_count)

    with driver.session(database="graph2024forecasted") as session:
        session.execute_write(create_person, name, id, paper_count)

In [5]:
def create_relationship(driver, name1, name2, weight):    
    def create_weighted_relationship(tx, name1, name2, weight):
        query = (
            f"MATCH (a:Concept {{id: '{name1}'}}), (b:Concept {{id: '{name2}'}})"
            f"CREATE (a)-[r:RELATED_TO {{weight: {weight}}}]->(b)"
            "RETURN r"
        )
        return tx.run(query)

    with driver.session(database="graph2024forecasted") as session:
        session.execute_write(create_weighted_relationship, name1, name2, weight)

In [6]:
# Load data
node_data = pd.read_csv('node_forecast_results_2024.csv')
edge_data = pd.read_csv('edge_forecast_results_2024.csv')

In [7]:
driver = GraphDatabase.driver(URI, auth=AUTH)

In [8]:
# Create nodes
for index, row in node_data.iterrows():
    create_node(driver, row['Concept'], row['ID'], row['Paper Count'])

# Create relationships
for index, row in edge_data.iterrows():
    create_relationship(driver, row['ID_A'], row['ID_B'], row['Common Paper Count'])

driver.close()